# Import Packages

In [3]:
from tensorflow.keras.applications.vgg16 import VGG16

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
%matplotlib inline

# Introduction

Clinical Dementia Ratings (CDR) are provided in the dataset with the following attributions:

    0 = no dementia, 0.5 = very mild AD, 1 = mild AD, 2 = moderate AD

For the purpose of analysis, all patients with a CDR of 0 are giving a diagnosis of 0, denoting cognitively healthy. CDR ratings above 0 are converted to 1, for an AD diagnosis, and binary classification.

diagnosis_qual={0.:'normal', 0.5:'AD', 1.:'AD', 2.:'AD' }


# Load Diagnosis

# Load MRI Images

In [3]:
# find all Data, e.g. in OASIS 1

# Plot Images

# Preprocess MRI Images

In [5]:
# grayscale
# reshape
# test_train_split

# Convolutional neural network

In [ ]:
#input_shape = (256,256,1) #(208,176,1)

model = Sequential()
model.add(Conv2D(100, kernel_size=(3, 3), strides=(10,10),
                 activation='sigmoid', padding ='same',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))
model.add(Conv2D(50, (3, 3), activation='sigmoid', strides=(5,5), padding ='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))
model.add(Conv2D(25, kernel_size=(3,3), activation='sigmoid', strides = (1,1), padding ='same'))
model.add(MaxPooling2D(pool_size=(1, 1), padding='valid'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

EarlyStopping = EarlyStopping(monitor='loss', 
                              min_delta=0, 
                              patience=10, 
                              verbose=0, 
                              mode='auto') # stop when loss no longer improving

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy', 'recall', 'precision'])

model.fit(x_train, y_train, 
          epochs=600, 
          batch_size=32, 
          callbacks=[EarlyStopping])
